# Generate URDF for station with bumps

Import modules:

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import numpy as np

Define helper functions:

In [ ]:
def add_bump_link(robot, index, radius, length):
    link = ET.SubElement(robot, 'link', attrib={'name': f'bumplink{index:02d}'})
    vis = ET.SubElement(link, 'visual')
    geo = ET.SubElement(vis, 'geometry')
    cyl = ET.SubElement(geo,
                        'mesh',
                        attrib={
                            'filename': 'disc.stl',
                            'scale': f'{radius:.3f} {radius:.3f} {2 * length:.3f}'
                        })
    mat = ET.SubElement(vis, 'material', attrib={'name': 'heritage-orange'})
    col = ET.SubElement(link, 'collision')
    geo = ET.SubElement(col, 'geometry')
    cyl = ET.SubElement(geo,
                        'mesh',
                        attrib={
                            'filename': 'disc.stl',
                            'scale': f'{radius:.3f} {radius:.3f} {2 * length:.3f}'
                        })
    ine = ET.SubElement(link, 'inertial')
    mas = ET.SubElement(ine, 'mass', attrib={'value': '1.0'})
    moi = ET.SubElement(ine, 'inertia', attrib={
        'ixx': '0',
        'ixy': '0',
        'ixz': '0',
        'iyy': '0',
        'iyz': '0',
        'izz': '0',
    })

def add_bump_joint(robot, index, theta, z):
    r = 20.
    x = r * np.sin(theta)
    y = -r * np.cos(theta)
    z = z
    roll = (np.pi / 2)
    pitch = 0
    yaw = theta
    joint = ET.SubElement(robot, 'joint', attrib={'name': f'bumpjoint{index:02d}', 'type': 'fixed'})
    par = ET.SubElement(joint, 'parent', attrib={'link': 'station'})
    chi = ET.SubElement(joint, 'child', attrib={'link': f'bumplink{index:02d}'})
    ori = ET.SubElement(
        joint,
        'origin',
        attrib={'xyz': f'{x:.3f} {y:.3f} {z:.3f}',
                'rpy': f'{roll:.3f} {pitch:.3f} {yaw:.3f}'},
    )

Print XML with bump links and joints for copy/paste into URDF:

In [ ]:
robot = ET.Element('robot')

num_bumps = 40
theta = np.linspace(0, 2 * np.pi, num_bumps + 2)
theta = theta[1:-1]
for i in range(num_bumps):
    add_bump_link(robot, i, 0.3, 0.02)
add_bump_link(robot, num_bumps, 1.5, 0.015)
for i in range(num_bumps):
    add_bump_joint(robot, i, theta[i] + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.5, 0.5))
add_bump_joint(robot, num_bumps, 0., 0.)

xmlstr = minidom.parseString(ET.tostring(robot)).toprettyxml(indent="  ")
print(xmlstr)